# XOR Trajectory when learning

_Aside: Should do [Working efficiently with jupyter lab](https://florianwilhelm.info/2018/11/working_efficiently_with_jupyter_lab/)_

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
#%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Fetch our tools:

In [ ]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NNBench
from nnvis import NNVis

___

## Setup
Build our `xor` net:

In [ ]:
net = Network()
net.extend(AffineLayer(2,2))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
net.extend(AffineLayer(2,1))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))

Make a test bench and a visualizer:

In [ ]:
bench = NNBench(net)
vis = NNVis(bench)

Prepare fixed training data for the learning process _[improve]_:

In [ ]:
dat = \
[(np.array([-1,-1]), np.array([-1])),
 (np.array([-1,1]), np.array([1])),
 (np.array([1,1]), np.array([-1])),
 (np.array([1,-1]), np.array([1]))]
dc = 0
amp= 1
temp = [(d[0]*amp/2+dc,d[1]*amp/2+dc) for d in dat]

bench.training_data = ((np.array([v[0] for v in temp]),
                        np.array([v[1] for v in temp])),)
bench.training_data_gen = bench.training_data_gen_fixed

Set the state to an ordinary example starting point, for consistent notebook behavior below. We also make it the checkpoint in the bench.

In [ ]:
net.set_state_from_vector(np.array([-0.88681521, -1.28596788,  0.3248974 , -2.33838503,  0.34761944,
       -0.94541789,  1.99448043,  0.38704839, -3.8844268 ]))
bench.checkpoint_net()

## The track that learning takes
Let us examine the trajectory in state space during learning, and the loss function.
Each learning iteration changes the net state. We can examine those deltas.
Questions:
1. Are there regimes of direction-of-change (DoC) in state space, or does the DoC wander chaotically?
1. What are the spectral characteristics of the DoC? Length characteristics?
1. How do the DoC characteristics relate to the loss function, and it's first difference?
1. How do these trajectories vary with learning rate? Are there clues in these to adapt the learning rate?
1. How do the trajectory characteristics vary across different starting nets?
1. How do these measures vary with the objective function of the learning process, that is, what you're trying to teach the net?
1. How do the different layers with learning state evolve? Do they settle at different times? How does an upstream layer change, as a consequence of learning, affect downstream layers? Down affect up?

In [ ]:
bench.rollback_net()
#bench.net.eta = 0.25
bench.net.eta = 0.4101
learned_track = bench.learn_track(5000)

Find the angles between trajectory steps, from
$$\mathbf {a} \cdot \mathbf {b} = \left\|\mathbf {a} \right\|\left\|\mathbf {b} \right\|\cos \theta \\
\cos \theta = \frac{\mathbf {a} \cdot \mathbf {b}}{\left\|\mathbf {a} \right\|\left\|\mathbf {b} \right\|} \\
$$
where $\mathbf {a}$ and $\mathbf {b}$ are a state-space trajectory step and the succeeding step respectively

In [ ]:
traja = bench.analyze_learning_track(learned_track)
vis.plot_trajectory(traja)

## Conclusions
We asked:
1. Are there regimes of direction-of-change (DoC) in state space, or does the DoC wander chaotically? \
    _**Answer:** Regimes exist. Often the initial direction is maintained (cos near 1) up to some region where the
    direction becomes chaotic, the loss improves, then falls into back-and-forth oscillations (cos near 0)
    perfecting the loss._
1. What are the spectral characteristics of the DoC? Length characteristics?
1. How do the DoC characteristics relate to the loss function, and it's first difference?
1. How do these trajectories vary with learning rate? Are there clues in these to adapt the learning rate?
1. How do the trajectory characteristics vary across different starting nets?
1. How do these measures vary with the objective function of the learning process, that is, what you're trying to teach the net?
1. How do the different layers with learning state evolve? Do they settle at different times? How does an upstream layer change, as a consequence of learning, affect downstream layers? Down affect up?

---

# Scratch

In [ ]:
assert False, "Stop notebook execution here if entering from above"

In [ ]:
bench.randomize_net()
t = bench.learn(1000)

In [ ]:
bench.net.state_vector()

In [ ]:
net = bench.net
net([[-.5, -.5], [-.5, .5]])

In [ ]:
net.layers

In [ ]:
from nnbench import Thing

In [ ]:
t = Thing(color='brown', weight=7)

In [ ]:
t.cow = 'moo'

In [ ]:
t.cow

In [ ]:
t.color

---

In [ ]:
# Boneyard

In [ ]:
assert False, "Stop notebook execution, the rest is scrap"

Wrangle the state-space trajectory and the losses into form.

In [ ]:
trajectory = np.vstack([v[0] for v in lt])
losses = np.vstack([v[1] for v in lt])

Take first differences, which represent the changes at each step

In [ ]:
traj_steps = np.diff(trajectory, axis=0)
loss_steps = np.diff(losses, axis=0)

In [ ]:
traj_steps[:5]

Find the L2 norm of the trajectory steps $\lVert traj \rVert$:

In [ ]:
traj_L2 = np.sqrt(np.einsum('...i,...i', traj_steps, traj_steps))

In [ ]:
len(traj_L2), traj_L2[:5], traj_L2[-5:]

Find the angles between trajectory steps, from
$$\mathbf {a} \cdot \mathbf {b} = \left\|\mathbf {a} \right\|\left\|\mathbf {b} \right\|\cos \theta \\
\cos \theta = \frac{\mathbf {a} \cdot \mathbf {b}}{\left\|\mathbf {a} \right\|\left\|\mathbf {b} \right\|} \\
$$
where $\mathbf {a}$ and $\mathbf {b}$ are a state-space trajectory step and the succeeding step respectively

Find $\mathbf {a} \cdot \mathbf {b}$:

In [ ]:
trajn_dot_nplus1 = np.einsum('...i,...i', traj_steps[:-1], traj_steps[1:])
trajn_dot_nplus1[:5], np.any(trajn_dot_nplus1 < 0)

Find $\left\|\mathbf {a} \right\|\left\|\mathbf {b} \right\|$:

In [ ]:
traj_cos_denom = np.multiply(traj_L2[:-1], traj_L2[1:])

This will be the divisor. Some entries may be zero, so we adapt

In [ ]:
len(traj_L2) - np.count_nonzero(traj_L2)

In [ ]:
np.equal(traj_L2, 0)

Find $\cos \theta$ by dividing, excluding division by zero:

In [ ]:
traj_cos = np.divide(trajn_dot_nplus1, traj_cos_denom, where=traj_cos_denom!=0.0)
traj_cos[:5], traj_cos[-5:], min(traj_cos), max(traj_cos)

In [ ]:
#traj_theta = np.arccos(traj_cos)
#traj_theta[:5], traj_theta[-5:]

In [ ]:
net = Network()
net.extend(AffineLayer(2,2))
#leak = 0
#net.extend(MapLayer(lambda x: (x*(1+leak/2)+abs(x)*(1-leak/2))/2, lambda d: [leak,1][1 if d>0 else 0]))
#net.extend(MapLayer(lambda x: max(0, np.sign(x)) * x, lambda d: max(0, np.sign(d))))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
net.extend(AffineLayer(2,1))
net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
#sigmoid = lambda x: 1/(np.exp(x)+1)
#net.extend(MapLayer(sigmoid, lambda d: sigmoid(d)*(1-sigmoid(d))))
#net.extend(MapLayer(lambda x: max(0, np.sign(x)) * x, lambda d: max(0, np.sign(d))))